In [1]:
!pip list | grep datasets
!pip list | grep sentencepiece
!pip list | grep transformers

datasets                      2.11.0
sentencepiece                 0.1.96
transformers                  4.28.1


In [2]:
# !pip install bitsandbytes
# !pip install loralib
# !pip install -q git+https://github.com/huggingface/peft.git

In [3]:
import os

import torch
from torch import nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers as T
from transformers import AutoTokenizer, AutoConfig
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, get_peft_model_state_dict

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so...


/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so.

In [4]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3 
LEARNING_RATE = 3e-4
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000

In [5]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


This model has some weights that should be kept in higher precision, you need to upgrade `accelerate` to properly deal with them (`pip install --upgrade accelerate`).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 model = LlamaForCausalLM.from_pretrained(                                                    │
│   2 │   "decapoda-research/llama-7b-hf",                                                         │
│   3 │   load_in_8bit=True,                                                                       │
│   4 │   device_map="auto",                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/transformers/modeling_utils.py:2846 in from_pretrained    │
│                                                                                                  │
│   2843 │   │                                                                                     │
│   2844 │   │   # Dispatch model with hooks on all devices if necessary                           │
│   2845 │   │   if device_map is not None:                                                        │
│ ❱ 2846 │   │   │   dispatch_model(model, device_map=device_map, offload_dir=offload_folder, off  │
│   2847 │   │                                                                                     │
│   2848 │   │   if output_loading_info:                                                           │
│   2849 │   │   │   if loading_info is None:                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: dispatch_model() got an unexpected keyword argument 'offload_index'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True,
)

In [ ]:
model = prepare_model_for_int8_training(model)

In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_perf_model(model, config)
tokenizer.pad_token_id = 0